In [1]:
import numpy as np
import time
import glob
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import image
from skimage.transform import resize
from skimage.io import imread
# from PIL import Image
# from keras.utils import to_categorical
from keras import Input
from keras import layers
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.regularizers import l2
from keras import Model
from keras import losses

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
# MacOS matplotlib kernel issue
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

Using TensorFlow backend.
/Users/tejas/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/tejas/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/tejas/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/tejas/anaconda3/lib/python3.7/site-packages/tenso

train_datagen = image.ImageDataGenerator(rescale=1./255)
train_gen = train_datagen.flow_from_directory('.', classes=['anime_face'], target_size = (128, 128), 
                                              batch_size = 32, color_mode='rgb')

In [2]:
# Global Constants
NOISE = (1,1,256)
IMAGE_SHAPE = (128,128,3)
# GAN_STEPS = int(140000 / train_gen.batch_size)
GAN_STEPS = 300
BATCH_SIZE = 32

In [3]:
def generator_model(noise=NOISE):
    gen_input = Input(shape=noise)
    generator = layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', kernel_initializer='glorot_uniform')(gen_input)
    generator = layers.BatchNormalization(momentum=0.5)(generator)
    generator = layers.LeakyReLU(alpha=0.2)(generator)
    
    generator = layers.Conv2DTranspose(filters=128, kernel_size=(4,4), strides=(2,2), padding='same', kernel_initializer='glorot_uniform')(generator)
    generator = layers.BatchNormalization(momentum=0.5)(generator)
    generator = layers.LeakyReLU(alpha=0.2)(generator)
    
    generator = layers.Conv2DTranspose(filters=128, kernel_size=(4,4), strides=(2,2), padding='same', kernel_initializer='glorot_uniform')(generator)
    generator = layers.BatchNormalization(momentum=0.5)(generator)
    generator = layers.LeakyReLU(alpha=0.2)(generator)
    
    generator = layers.Conv2DTranspose(filters=64, kernel_size=(5,5), strides=(2,2), padding='same', kernel_initializer='glorot_uniform')(generator)
    generator = layers.BatchNormalization(momentum=0.5)(generator)
    generator = layers.LeakyReLU(alpha=0.2)(generator)
    
    generator = layers.Conv2DTranspose(filters=64, kernel_size=(4,4), strides=(2,2), padding='same', kernel_initializer='glorot_uniform')(generator)
    generator = layers.BatchNormalization(momentum=0.5)(generator)
    generator = layers.LeakyReLU(alpha=0.2)(generator)
    
#     generator = layers.Conv2D(filters=128, kernel_size=(4,4), strides=(1,1), padding='same', kernel_initializer='glorot_uniform')(generator)
#     generator = layers.BatchNormalization(momentum=0.5)(generator)
#     generator = layers.LeakyReLU(alpha=0.2)(generator)
    
    generator = layers.Conv2DTranspose(filters=32, kernel_size=(4,4), strides=(2,2), padding='same', kernel_initializer='glorot_uniform')(generator)
    generator = layers.BatchNormalization(momentum=0.5)(generator)
    generator = layers.LeakyReLU(alpha=0.2)(generator)
    
    generator = layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same', kernel_initializer='glorot_uniform')(generator)
    generator = layers.BatchNormalization(momentum=0.5)(generator)
    generator = layers.LeakyReLU(alpha=0.2)(generator)
    
    generator = layers.Conv2DTranspose(filters=16, kernel_size=(4,4), strides=(2,2), padding='same', kernel_initializer='glorot_uniform')(generator)
    generator = layers.BatchNormalization(momentum=0.5)(generator)
    generator = layers.LeakyReLU(alpha=0.2)(generator)
    
    generator = layers.Conv2DTranspose(filters=3, kernel_size=(4,4), strides=(2,2), activation='tanh', padding='same', kernel_initializer='glorot_uniform')(generator)
    
    model = Model(inputs=gen_input, outputs=generator)
    model.compile(optimizer=RMSprop(lr=1e-5, clipvalue=1.0, decay=1e-8), loss=losses.binary_crossentropy, metrics=['accuracy'])
    
    return model

In [4]:
def discriminator_model(image_shape=IMAGE_SHAPE):
    disc_input = Input(shape=image_shape)
    discriminator = layers.Conv2D(filters=32, kernel_size=(4,4), padding='same', strides=(2,2), kernel_initializer='glorot_uniform')(disc_input)
#     discriminator = layers.BatchNormalization(momentum=0.5)(discriminator)
    discriminator = layers.LeakyReLU(alpha=0.2)(discriminator)
    
    discriminator = layers.Conv2D(filters=64, kernel_size=(4,4), padding='same', strides=(2,2), kernel_initializer='glorot_uniform', kernel_regularizer=l2(1e-3))(discriminator)
#     discriminator = layers.BatchNormalization(momentum=0.5)(discriminator)
    discriminator = layers.LeakyReLU(alpha=0.2)(discriminator)
    
    discriminator = layers.Conv2D(filters=128, kernel_size=(4,4), padding='same', strides=(2,2), kernel_initializer='glorot_uniform', kernel_regularizer=l2(1e-3))(discriminator)
    discriminator = layers.BatchNormalization(momentum=0.5)(discriminator)
    discriminator = layers.LeakyReLU(alpha=0.2)(discriminator)
    
    discriminator = layers.Conv2D(filters=128, kernel_size=(4,4), padding='same', strides=(2,2), kernel_initializer='glorot_uniform', kernel_regularizer=l2(1e-3))(discriminator)
    discriminator = layers.BatchNormalization(momentum=0.5)(discriminator)
    discriminator = layers.LeakyReLU(alpha=0.2)(discriminator)
    
    discriminator = layers.Conv2D(filters=256, kernel_size=(4,4), padding='same', strides=(2,2), kernel_initializer='glorot_uniform', kernel_regularizer=l2(1e-3))(discriminator)
    discriminator = layers.BatchNormalization(momentum=0.5)(discriminator)
    discriminator = layers.LeakyReLU(alpha=0.2)(discriminator)
    
    discriminator = layers.Flatten()(discriminator)
#     discriminator = layers.Dense(128)(discriminator)
#     discriminator = layers.LeakyReLU(alpha=0.2)(discriminator)
    discriminator = layers.Dense(1, activation='sigmoid')(discriminator)
    
    model = Model(inputs=disc_input, outputs=discriminator)
    model.compile(optimizer=RMSprop(lr=1e-4, clipvalue=1.0, decay=1e-8), loss=losses.binary_crossentropy, metrics=['accuracy'])
    
    return model

In [5]:
gen_model = generator_model(NOISE)
gen_model.summary()








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 1, 256)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 1, 256)         590080    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 1, 256)         1024      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 1, 1, 256)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 2, 2, 128)         524416    
_________________________________________________________________
batch_normalization_2 (Batch (None, 2, 2, 128)         512       
_________________________________________

In [6]:
disc_model = discriminator_model()
disc_model.summary()
disc_model.trainable = False

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 32)        1568      
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 64)        32832     
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 128)       131200    
_________________________________________________________________
batch_normalization_9 (Batch (None, 16, 16, 128)       512       
__________

In [7]:
gan_gen_input = Input(shape=NOISE)
gan_gen = gen_model(gan_gen_input)
gan_dis = disc_model(gan_gen)

gan_model = Model(inputs=gan_gen_input, outputs=gan_dis)
gan_model.compile(optimizer=RMSprop(lr=1e-5, clipvalue=1.0, decay=1e-8), loss=losses.binary_crossentropy, metrics=['accuracy'])
gan_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1, 1, 256)         0         
_________________________________________________________________
model_1 (Model)              (None, 128, 128, 3)       1701139   
_________________________________________________________________
model_2 (Model)              (None, 1)                 958561    
Total params: 2,659,700
Trainable params: 1,699,699
Non-trainable params: 960,001
_________________________________________________________________


In [8]:
def save_fig(predicted, current_time):
    num_images = predicted.shape[0]
    fig = plt.figure(figsize=(15,7))
    columns = 8
    rows = np.ceil(num_images / columns)
    for i in range(num_images):
        fig.add_subplot(rows, columns, i+1)
        my_image = (predicted[i] * 255).astype(np.uint8) / 255
        plt.imshow(my_image)
    plt.savefig('./GeneratedFigures/image_'+current_time+'.jpg', bbox_inches = 'tight', pad_inches = 0.1)
#     plt.show(block=True)

In [9]:
def get_real_images(batch_size=BATCH_SIZE, image_shape=IMAGE_SHAPE, data_search_pattern='./anime_face/00[5-9]*'):
    batch_image_shape = (batch_size,) + image_shape
    batch_images = np.empty(batch_image_shape, dtype=np.float32)
    all_images = glob.glob(data_search_pattern)
    chosen_images = np.random.choice(all_images, batch_size)
    for index, file in enumerate(chosen_images):
        my_image = resize(imread(file), IMAGE_SHAPE[:-1])
#         my_image = np.asarray(my_image) / 255
        batch_images[index,...] = my_image
    return batch_images

In [10]:
for file in glob.glob('./GeneratedFigures/*'):
    if file.endswith('.jpg'):
        os.remove(file)
for file in glob.glob('./GANModels/*'):
    if file.endswith('.h5'):
        os.remove(file)

In [11]:
with open('log.csv', 'w') as log:
    log.write('Step,DiscLoss,DiscAcc,GANLoss,GANAcc\n')
for step in range(GAN_STEPS):
    print('**************************************')
    print()
    print('               Step: ', step)
    print()
    print('**************************************')
    
    current_time = time.strftime('%Y_%m_%d_%H_%M_%S', time.localtime())
    print("Creating noise")
    gen_noise = np.random.normal(loc=0, scale=1, size=(BATCH_SIZE,)+NOISE)
    print("Predicting noise")
    created_faces = gen_model.predict(gen_noise)
    if ((step % 1) == 0):
        save_fig(created_faces, current_time)
    
    # Merge real and fake data
    real_faces = get_real_images()
#     print("Batch Index: ", train_gen.batch_index)
    combined_data = np.concatenate([real_faces, created_faces])
    combined_labels = np.concatenate([np.ones((BATCH_SIZE, 1), dtype=np.int).ravel(), np.zeros((BATCH_SIZE, 1), dtype=np.int).ravel()])
    
    # Train Discriminator
    disc_model.trainable = True
    gen_model.trainable = False
    print("Training Discriminator")
    disc_metrics = disc_model.train_on_batch(combined_data, combined_labels)
    print("disc_metrics")
    print(disc_metrics)

#     # Merge real and fake data
#     real_faces, real_labels = train_gen.next()
#     print("Batch Index: ", train_gen.batch_index)
#     fake_labels = np.zeros((BATCH_SIZE, 1))
    
#     encoder = LabelEncoder()
#     real_labels = encoder.fit_transform(real_labels)
#     fake_labels = encoder.fit_transform(fake_labels)
    
    
#     # Train Discriminator
#     disc_model.trainable = True
#     gen_model.trainable = False
#     print("Training Real Discriminator")
#     disc_metrics = disc_model.train_on_batch(real_faces, real_labels)
#     print("disc_metrics")
#     print(disc_metrics)
#     print("Training Fake Discriminator")
#     disc_metrics = disc_model.train_on_batch(created_faces, fake_labels)
#     print("disc_metrics")
#     print(disc_metrics)


    
    # Train GAN
    gen_model.trainable = True
    disc_model.trainable = False
    print("Creating GAN Noise")
    gan_noise = np.random.normal(loc=0, scale=1, size=(BATCH_SIZE,)+NOISE)
    gan_labels = np.ones((BATCH_SIZE, 1), dtype=np.int).ravel()
    print("Training GAN")
    gan_metrics = gan_model.train_on_batch(gan_noise, gan_labels)
    print("gan_metrics")
    print(gan_metrics)
    
    # Append Log
    with open('log.csv', 'a') as log:
        log.write('%d,%f,%f,%f,%f\n' % (step, disc_metrics[0], disc_metrics[1], gan_metrics[0], gan_metrics[1]))
    if ((step % 50) == 0):
        gan_model.save('./GANModels/model_'+current_time+".h5")

**************************************

               Step:  0

**************************************
Creating noise
Predicting noise
Training Discriminator
disc_metrics
[1.0260196, 0.734375]
Creating GAN Noise
Training GAN
gan_metrics
[0.64720136, 0.96875]
**************************************

               Step:  1

**************************************
Creating noise
Predicting noise
Training Discriminator
disc_metrics
[0.5484728, 0.984375]
Creating GAN Noise
Training GAN
gan_metrics
[1.2979372, 0.3125]
**************************************

               Step:  2

**************************************
Creating noise
Predicting noise
Training Discriminator
disc_metrics
[0.43671843, 1.0]
Creating GAN Noise
Training GAN
gan_metrics
[1.2420629, 0.4375]
**************************************

               Step:  3

**************************************
Creating noise
Predicting noise
Training Discriminator
disc_metrics
[0.43169984, 1.0]
Creating GAN Noise
Training GAN
gan_me

KeyboardInterrupt: 